###Pipeline de clasificación

In [67]:
#Importacion de libreririas

from xgboost import XGBClassifier
import pandas as pd
import kagglehub
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd


In [68]:
#######Importacion de datos######
# Download latest version
path = kagglehub.dataset_download("stefanouccelli/tesis-1")

print("Path to dataset files:", path)
# List files in the downloaded directory
files_in_path = os.listdir(path)
print("Files in the downloaded directory:", files_in_path)

# Construct the full path to the downloaded file
old_file_path = os.path.join(path, files_in_path[0])

# Construct the new file path in a writable directory
new_file_path_in_writable = os.path.join( files_in_path[0])

# Copy the file to the writable directory
shutil.copyfile(old_file_path, new_file_path_in_writable)

print(f"Copied '{old_file_path}' to '{new_file_path_in_writable}'")

# Construct the final file path after removing '.crdownload'
final_file_path = new_file_path_in_writable.replace('.crdownload', '')

# Rename the copied file
os.rename(new_file_path_in_writable, final_file_path)

print(f"Renamed '{new_file_path_in_writable}' to '{final_file_path}'")

# Load the renamed file into a pandas DataFrame
df = pd.read_csv(final_file_path)

# Display the first few rows of the DataFrame


Path to dataset files: C:\Users\el_me\.cache\kagglehub\datasets\stefanouccelli\tesis-1\versions\1
Files in the downloaded directory: ['PS_20174392719_1491204439457_log (1).csv.crdownload']
Copied 'C:\Users\el_me\.cache\kagglehub\datasets\stefanouccelli\tesis-1\versions\1\PS_20174392719_1491204439457_log (1).csv.crdownload' to 'PS_20174392719_1491204439457_log (1).csv.crdownload'
Renamed 'PS_20174392719_1491204439457_log (1).csv.crdownload' to 'PS_20174392719_1491204439457_log (1).csv'


In [105]:
df[df['isFraud'] == 1].sample()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2382150,197,TRANSFER,486060.16,C813852044,486060.16,0.0,C34034576,0.0,0.0,1.0,0.0


In [69]:
df_prueba = df.iloc[15005:15012]
df_prueba

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
15005,8,PAYMENT,20486.90,C329162773,200755.0,180268.10,M1348027050,0.0,0.00,0.0,0.0
15006,8,CASH_IN,58656.72,C641519842,902359.0,961015.72,C1672788422,0.0,114194.51,0.0,0.0
15007,8,CASH_OUT,234151.90,C1801715006,251.0,0.00,C1186256103,110703.0,4999613.43,0.0,0.0
15008,8,TRANSFER,266086.17,C228288079,304381.0,38294.83,C47249488,380.0,0.00,0.0,0.0
15009,8,PAYMENT,10450.65,C390936900,11034.0,583.35,M1463177359,0.0,0.00,0.0,0.0
15010,8,PAYMENT,6698.24,C600366426,50621.0,43922.76,M1974955634,0.0,0.00,0.0,0.0
15011,8,PAYMENT,5390.73,C198110873,996.0,0.00,M1855550791,0.0,0.00,0.0,0.0


In [76]:
dicc_percentage = { 
    'CASH_OUT': 0.503023,
    'TRANSFER': 0.496977,
    'PAYMENT': 0,
    'DEBIT': 0,
    'CASH_IN': 0
}

epsilon = 1e-10
bins = [0.0, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95,   float('inf')]
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class IngenieriaTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df_t = X.copy()
        
        # Aplicar transformaciones
        df_t['type_percentage'] = df_t['type'].map(dicc_percentage)
        df_t['amount'] = np.sqrt(df_t['amount'])
        df_t['log_amount'] = np.log1p(df_t['amount'])
        df_t['newbalanceDestSqrt'] = np.sqrt(df_t['newbalanceDest'])
        df_t['vacied_account'] = np.where(
            ((df_t['type'] == 'TRANSFER') & (df_t['newbalanceDest'] == 0)), 1, 0
        )
        df_t['balance_Change'] = np.abs((df_t['newbalanceDest'] - df_t['oldbalanceDest']) / (df_t['oldbalanceDest'] + epsilon)).clip(0, 100)
        df_t['balance_Change_Origin'] = np.abs((df_t['newbalanceOrig'] - df_t['oldbalanceOrg']) / (df_t['oldbalanceOrg'] + epsilon)).clip(0, 100)
        
        df_t['balance_Change_Range'] = pd.cut(df_t['balance_Change'], bins=bins, labels=labels, include_lowest=True, right=False)
        df_t['balance_Change_Origin_Range'] = pd.cut(df_t['balance_Change_Origin'], bins=bins, labels=labels, include_lowest=True, right=False)
        
        # Convertir a entero
        df_t['balance_Change_Range'] = df_t['balance_Change_Range'].astype('int')
        df_t['balance_Change_Origin_Range'] = df_t['balance_Change_Origin_Range'].astype('int')
        
        # Seleccionar columnas finales
        return df_t[['type_percentage', 'log_amount', 'vacied_account',
                     'balance_Change_Range', 'newbalanceDestSqrt',
                     'balance_Change_Origin_Range', 'step']]



In [86]:
df = df.dropna(subset=['isFraud'])  # eliminar NaNs en la variable objetivo
X = df.drop(columns=['isFraud'])
y = df['isFraud']

In [87]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=123, stratify=y
)

In [88]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

pipeline = Pipeline(steps=[
    ('ingenieria', IngenieriaTransformer()),
    ('classifier', XGBClassifier(
        eval_metric='error',
        random_state=42,
        n_jobs=-1,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=150,
        objective='binary:logistic',
        colsample_bytree=1,
        subsample=0.9
    ))
])

In [89]:
pipeline.fit(X_train, y_train)

y_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred_thresh = (y_proba > 0.4).astype(int)

print("Recall con umbral:", recall_score(y_test, y_pred_thresh))
print("F1 con umbral:", f1_score(y_test, y_pred_thresh))
print("Precisión con umbral:", precision_score(y_test, y_pred_thresh))

Recall con umbral: 0.694591728525981
F1 con umbral: 0.8022045315370484
Precisión con umbral: 0.9492753623188406


In [90]:
import joblib
joblib.dump(pipeline, "fraud_pipeline.pkl")

['fraud_pipeline.pkl']

In [106]:
from flask import Flask, request, jsonify, render_template_string
import joblib
import pandas as pd

app = Flask(__name__)
model = joblib.load("fraud_pipeline.pkl")

@app.route("/", methods=["GET"])
def home():
    return "API de detección de fraude funcionando 🚀"

@app.route("/form", methods=["GET"])
def form():
    html_form = """
    <!DOCTYPE html>
    <html lang="es">
    <head>
      <meta charset="UTF-8">
      <title>Fraude Predictor</title>
    </head>
    <body>
      <h2>Formulario de predicción de fraude</h2>
      <form id="fraudForm">
        <label>Tipo:</label>
        <select name="type">
          <option value="TRANSFER">TRANSFER</option>
          <option value="CASH_OUT">CASH_OUT</option>
        </select><br><br>

        <label>Monto:</label>
        <input type="number" name="amount" required><br><br>

        <label>Saldo destino nuevo:</label>
        <input type="number" name="newbalanceDest" required><br><br>

        <label>Saldo destino anterior:</label>
        <input type="number" name="oldbalanceDest" required><br><br>

        <label>Saldo origen nuevo:</label>
        <input type="number" name="newbalanceOrig" required><br><br>

        <label>Saldo origen anterior:</label>
        <input type="number" name="oldbalanceOrg" required><br><br>

        <label>Step:</label>
        <input type="number" name="step" required><br><br>

        <button type="submit">Predecir</button>
      </form>

      <h3 id="result"></h3>

      <script>
        document.getElementById("fraudForm").addEventListener("submit", function(e) {
          e.preventDefault();
          const formData = new FormData(e.target);
          const data = Object.fromEntries(formData.entries());

          // Convertir valores numéricos a tipo número
          data.amount = parseFloat(data.amount);
          data.newbalanceDest = parseFloat(data.newbalanceDest);
          data.oldbalanceDest = parseFloat(data.oldbalanceDest);
          data.newbalanceOrig = parseFloat(data.newbalanceOrig);
          data.oldbalanceOrg = parseFloat(data.oldbalanceOrg);
          data.step = parseInt(data.step);

          fetch("/predict", {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify(data)
          })
          .then(res => res.json())
          .then(res => {
            if (res.error) {
              document.getElementById("result").innerText = "Error: " + res.error;
            } else {
              document.getElementById("result").innerText =
                `Predicción: ${res.prediction} | Probabilidad: ${res.probability.toFixed(4)}`;
            }
          });
        });
      </script>
    </body>
    </html>
    """
    return render_template_string(html_form)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()

        # Convertir a DataFrame
        df_input = pd.DataFrame([data])

        # Predecir con el pipeline
        y_proba = model.predict_proba(df_input)[:, 1]
        y_pred = (y_proba > 0.4).astype(int)

        return jsonify({
            "prediction": int(y_pred[0]),
            "probability": float(y_proba[0])
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Nov/2025 08:33:46] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:50:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:51:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:51:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:51:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:51:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 08:51:00] "POST /predict HTTP/1.